# Anticipez les besoins en consommation électrique de bâtiments
Nous travaillons pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, notre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation individuelle. Des relevés minutieux ont été effectués en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, nous voulons tenter de prédire les émissions de CO2 et la consommation totale d’énergie des bâtiments pour lesquels elles n’ont pas encore été mesurées. </br>
Dans ce notebook, nous testerons différents modèles de prédiction de la consommation d'énergie des bâtiments de la ville de Seattle.

# Sommaire :
- **Partie 1:** <a href="#C1">Importation des données</a>
- **Partie 2:** <a href="#C2">Méthodes de Régression</a>
    - <a href="#C3">Régressions naïves</a>
    - <a href="#C4">Régression Linéaire</a>
    - <a href="#C5">Régression Ridge</a>
    - <a href="#C6">Régression Lasso</a>
    - <a href="#C7">Régression ElasticNet</a>
- **Partie 3:** <a href="#C8">Méthodes ensemblistes</a>
    - <a href="#C9">Bagging</a>
    - <a href="#C10">Random Forest</a>
    - <a href="#C11">Gradient Boosting</a>
- **Partie 4:** <a href="#C12">Méthodes non linéaires</a>
    - <a href="#C13"> SVM à noyau</a>
    - <a href="#C14">Régression ridge à noyau</a>
- **Partie 5:** <a href="#C15">Sélection du modèle de prédiction</a>

# <a name="C1">Partie 1: Importation des données</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import datasets
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, LassoCV, Lasso, ElasticNetCV, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.dummy import DummyRegressor
from sklearn import preprocessing
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

pd.set_option('display.max_columns', 100)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
#Jeu de données nettoyé sans transformation
data_original = pd.read_csv('dataNettoye.csv',sep=',',encoding='utf-8')
data = data_original.copy()
data.head()

,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,ENERGYSTARScore,SiteEnergyUse(kBtu),GHGEmissions(MetricTonsCO2e),BuildingAge,BuildingType_Multifamily HR (10+),BuildingType_Multifamily LR (1-4),BuildingType_Multifamily MR (5-9),BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Distribution Center\n,PrimaryPropertyType_High-Rise Multifamily,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mid-Rise Multifamily,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Non-Refrigerated Warehouse,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Residence Hall/Dormitory,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Restaurant\n,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Self-Storage Facility\n,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Supermarket/Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,...,SecondLargestPropertyUseType_Repair Services (Vehicle,SecondLargestPropertyUseType_Residence Hall/Dormitory,SecondLargestPropertyUseType_Restaurant,SecondLargestPropertyUseType_Retail Store,SecondLargestPropertyUseType_Self-Storage Facility,SecondLargestPropertyUseType_Senior Care Community,SecondLargestPropertyUseType_Shoe,SecondLargestPropertyUseType_Social/Meeting Hall,SecondLargestPropertyUseType_Strip Mall,SecondLargestPropertyUseType_Supermarket/Grocery Store,SecondLargestPropertyUseType_Swimming Pool,SecondLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_Financial Office,ThirdLargestPropertyUseType_Locksmith,ThirdLargestPropertyUseType_Manufacturing/Industrial Plant,ThirdLargestPropertyUseType_Medical Office,ThirdLargestPropertyUseType_Movie Theater,ThirdLargestPropertyUseType_Multifamily Housing,ThirdLargestPropertyUseType_Museum,ThirdLargestPropertyUseType_NoThirdUse,ThirdLargestPropertyUseType_Non-Refrigerated Warehouse,ThirdLargestPropertyUseType_Office,ThirdLargestPropertyUseType_Other,ThirdLargestPropertyUseType_Other - Education,ThirdLargestPropertyUseType_Other - Entertainment/Public Assembly,ThirdLargestPropertyUseType_Other - Lodging/Residential,ThirdLargestPropertyUseType_Other - Mall,ThirdLargestPropertyUseType_Other - Public Services,ThirdLargestPropertyUseType_Other - Recreation,ThirdLargestPropertyUseType_Other - Restaurant/Bar,ThirdLargestPropertyUseType_Other - Services,ThirdLargestPropertyUseType_Other - Technology/Science,ThirdLargestPropertyUseType_Outpatient Rehabilitation/Physical Therapy,ThirdLargestPropertyUseType_Parking,ThirdLargestPropertyUseType_Personal Services (Health/Beauty,ThirdLargestPropertyUseType_Pre-school/Daycare,ThirdLargestPropertyUseType_Refrigerated Warehouse,ThirdLargestPropertyUseType_Repair Services (Vehicle,ThirdLargestPropertyUseType_Residence Hall/Dormitory,ThirdLargestPropertyUseType_Restaurant,ThirdLargestPropertyUseType_Retail Store,ThirdLargestPropertyUseType_Self-Storage Facility,ThirdLargestPropertyUseType_Senior Care Community,ThirdLargestPropertyUseType_Shoe,ThirdLargestPropertyUseType_Social/Meeting Hall,ThirdLargestPropertyUseType_Supermarket/Grocery Store,ThirdLargestPropertyUseType_Swimming Pool,ThirdLargestPropertyUseType_Vocational School,ThirdLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_etc)
0,1927,1,1,88434,65.0,6981428.0,249.43,88,0,0,0,1,0,0,0,0,0

In [3]:
#Jeu de données nettoyé avec transformations
dataTransfo_original = pd.read_csv('dataNettoyeTransforme.csv',sep=',',encoding='utf-8')
dataLog = dataTransfo_original.copy()
dataLog.head()

,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,ENERGYSTARScore,SiteEnergyUse(kBtu),GHGEmissions(MetricTonsCO2e),BuildingAge,BuildingType_Multifamily HR (10+),BuildingType_Multifamily LR (1-4),BuildingType_Multifamily MR (5-9),BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Distribution Center\n,PrimaryPropertyType_High-Rise Multifamily,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mid-Rise Multifamily,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Non-Refrigerated Warehouse,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Residence Hall/Dormitory,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Restaurant\n,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Self-Storage Facility\n,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Supermarket/Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,...,SecondLargestPropertyUseType_Repair Services (Vehicle,SecondLargestPropertyUseType_Residence Hall/Dormitory,SecondLargestPropertyUseType_Restaurant,SecondLargestPropertyUseType_Retail Store,SecondLargestPropertyUseType_Self-Storage Facility,SecondLargestPropertyUseType_Senior Care Community,SecondLargestPropertyUseType_Shoe,SecondLargestPropertyUseType_Social/Meeting Hall,SecondLargestPropertyUseType_Strip Mall,SecondLargestPropertyUseType_Supermarket/Grocery Store,SecondLargestPropertyUseType_Swimming Pool,SecondLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_Financial Office,ThirdLargestPropertyUseType_Locksmith,ThirdLargestPropertyUseType_Manufacturing/Industrial Plant,ThirdLargestPropertyUseType_Medical Office,ThirdLargestPropertyUseType_Movie Theater,ThirdLargestPropertyUseType_Multifamily Housing,ThirdLargestPropertyUseType_Museum,ThirdLargestPropertyUseType_NoThirdUse,ThirdLargestPropertyUseType_Non-Refrigerated Warehouse,ThirdLargestPropertyUseType_Office,ThirdLargestPropertyUseType_Other,ThirdLargestPropertyUseType_Other - Education,ThirdLargestPropertyUseType_Other - Entertainment/Public Assembly,ThirdLargestPropertyUseType_Other - Lodging/Residential,ThirdLargestPropertyUseType_Other - Mall,ThirdLargestPropertyUseType_Other - Public Services,ThirdLargestPropertyUseType_Other - Recreation,ThirdLargestPropertyUseType_Other - Restaurant/Bar,ThirdLargestPropertyUseType_Other - Services,ThirdLargestPropertyUseType_Other - Technology/Science,ThirdLargestPropertyUseType_Outpatient Rehabilitation/Physical Therapy,ThirdLargestPropertyUseType_Parking,ThirdLargestPropertyUseType_Personal Services (Health/Beauty,ThirdLargestPropertyUseType_Pre-school/Daycare,ThirdLargestPropertyUseType_Refrigerated Warehouse,ThirdLargestPropertyUseType_Repair Services (Vehicle,ThirdLargestPropertyUseType_Residence Hall/Dormitory,ThirdLargestPropertyUseType_Restaurant,ThirdLargestPropertyUseType_Retail Store,ThirdLargestPropertyUseType_Self-Storage Facility,ThirdLargestPropertyUseType_Senior Care Community,ThirdLargestPropertyUseType_Shoe,ThirdLargestPropertyUseType_Social/Meeting Hall,ThirdLargestPropertyUseType_Supermarket/Grocery Store,ThirdLargestPropertyUseType_Swimming Pool,ThirdLargestPropertyUseType_Vocational School,ThirdLargestPropertyUseType_Worship Facility,ThirdLargestPropertyUseType_etc)
0,7.564238,1,1,11.390023,4.189655,15.758764,5.523179,4.4886

On a vu dans le notebook d'étude des prédictions d'émissions de CO2 que la variable `ENERGYSTARScore` n'apportait pas forcément de gain notable dans la prédiction mais qu'elle conduisait à des modèles plus robustes. On va donc la conserver,pour entraîner nos modèles de prédiction des consommations d'énergie. On conserve également la variable `GHGEmissions(MetricTonsCO2e)` que l'on peut au besoin estimer avec le modèle prédiction choisi auparavant.

On peut à présent préparer notre jeu d'entraînement qui servira pour entrainer les différents modèles que l'on va traiter, et notre jeu de test qui servira à comparer les performances de généralisation des différents modèles.

In [4]:
#Pour le jeu non transformé

#Jeux d'entraînement et de test
X = data.drop(['SiteEnergyUse(kBtu)'], axis=1)
y = data['SiteEnergyUse(kBtu)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#Standardisation des données d'entraînement
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

In [5]:
#Pour le jeu transformé

#Jeux d'entraînement et de test
X_log = dataLog.drop(['SiteEnergyUse(kBtu)'], axis=1)
y_log = dataLog['SiteEnergyUse(kBtu)']
X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log, y_log, test_size = 0.2)

#Standardisation des données d'entraînement
std_scale = preprocessing.StandardScaler().fit(X_log_train)
X_log_train_std = std_scale.transform(X_log_train)
X_log_test_std = std_scale.transform(X_log_test)

On prépare le tableau final de comparaison des modèles.

In [6]:
listeModeles = []

listeR2, listeR2_log = [], []
listeR2_train, listeR2_log_train = [], []
listeMSE, listeMSE_log = [], []
listeTemps, listeTemps_log = [], []

# <a name="C2">Partie 2: Méthodes de Régression</a>
## <a name="C3">2.1: Régressions naïves</a> 
On va commencer par les modèles les plus simples possibles, à savoir prédire à chaque fois la valeur moyenne dans un premier temps, puis la valeur médiane dans un second temps.
### 2.1.1: Jeu sans transformation:

In [7]:
listeModeles.append('Dummy_mean')
listeModeles.append('Dummy_median')

In [8]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train_std, y_train)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test_std)
time_mean = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy moyenne): {:.4f}".format(lm_dummy_mean.score(X_train_std, y_train)))
listeR2_train.append(lm_dummy_mean.score(X_train_std, y_train))

#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_train_std, y_train)
y_predict_dummy_median = lm_dummy_median.predict(X_test_std)
time_median = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy médiane): {:.4f}".format(lm_dummy_median.score(X_train_std, y_train)))
listeR2_train.append(lm_dummy_median.score(X_train_std, y_train))

r2_score train (dummy moyenne): 0.0000
r2_score train (dummy médiane): -0.0311


In [9]:
#Score du modèle sur le test_set
print("Mean squared error (dummy moyenne): {:.4f}".format(mean_squared_error(y_test, y_predict_dummy_mean)))
print("Mean squared error (dummy médiane): {:.4f}".format(mean_squared_error(y_test, y_predict_dummy_median)))
  
print("r2_score (dummy moyenne): {:.4f}".format(r2_score(y_test, y_predict_dummy_mean)))
print("r2_score (dummy médiane): {:.4f}".format(r2_score(y_test, y_predict_dummy_median)))

print("Temps de calcul (dummy moyenne): {:.4f}".format(time_mean))
print("Temps de calcul (dummy médiane): {:.4f}".format(time_median))

Mean squared error (dummy moyenne): 262612983505330.8125
Mean squared error (dummy médiane): 275283407856870.0000
r2_score (dummy moyenne): -0.0003
r2_score (dummy médiane): -0.0485
Temps de calcul (dummy moyenne): 0.0022
Temps de calcul (dummy médiane): 0.0009


In [10]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict_dummy_mean),4))
listeMSE.append(round(mean_squared_error(y_test, y_predict_dummy_median),4))

listeR2.append(round(r2_score(y_test, y_predict_dummy_mean),4))
listeR2.append(round(r2_score(y_test, y_predict_dummy_median),4))

listeTemps.append(round(time_mean,4))
listeTemps.append(round(time_median,4))

Le jeu de données est plus complexe que les approches naïves de prédiction de la moyenne et de la médiane.

### 2.1.2: Jeu avec transformations
Regardons si les transformations logarithmiques réalisées dans la partie nettoyage peuvent améliorer les capacités de généralisation de ces premières approches naïves.

In [11]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_log_train_std, y_log_train)
y_log_predict_dummy_mean = lm_dummy_mean.predict(X_log_test_std)
time_mean = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy moyenne): {:.4f}".format(lm_dummy_mean.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lm_dummy_mean.score(X_log_train_std, y_log_train))

#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_log_train_std, y_log_train)
y_log_predict_dummy_median = lm_dummy_median.predict(X_log_test_std)
time_median = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train (dummy médiane): {:.4f}".format(lm_dummy_median.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lm_dummy_median.score(X_log_train_std, y_log_train))

r2_score train (dummy moyenne): 0.0000
r2_score train (dummy médiane): -0.0072


In [12]:
#Score du modèle sur le test_set
print("Mean squared error (dummy moyenne): {:.4f}".format(mean_squared_error(y_log_test, y_log_predict_dummy_mean)))
print("Mean squared error (dummy médiane): {:.4f}".format(mean_squared_error(y_log_test, y_log_predict_dummy_median)))
  
print("r2_score (dummy moyenne): {:.4f}".format(r2_score(y_log_test, y_log_predict_dummy_mean)))
print("r2_score (dummy médiane): {:.4f}".format(r2_score(y_log_test, y_log_predict_dummy_median)))

print("Temps de calcul (dummy moyenne): {:.4f}".format(time_mean))
print("Temps de calcul (dummy médiane): {:.4f}".format(time_median))

Mean squared error (dummy moyenne): 1.8750
Mean squared error (dummy médiane): 1.9011
r2_score (dummy moyenne): -0.0013
r2_score (dummy médiane): -0.0153
Temps de calcul (dummy moyenne): 0.0010
Temps de calcul (dummy médiane): 0.0013


In [13]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict_dummy_mean),4))
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict_dummy_median),4))

listeR2_log.append(round(r2_score(y_log_test, y_log_predict_dummy_mean),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict_dummy_median),4))

listeTemps_log.append(round(time_mean,4))
listeTemps_log.append(round(time_median,4))

On n'obtient pas de meilleurs résultats.

## <a name="C4">Partie 2.2: Régression Linéaire</a>
Les premières approches naïves n'étant pas satisfaisantes, on va complexifier un peu le modèle.

Notre démarche, qui sera la même pour tous les prochains modèles, est la suivante:
- Trouver les meilleurs hyperparamètres (lorqu'il y en a) par recherche sur grille
- Réaliser une validation croisée sur le jeu d'entraînement pour avoir une idée des capacités de généralisation du modèle et pouvoir contrôler l'éventuel sur/sous-apprentissage
- Entraîner le modèle sur le jeu d'entraînement
- Evaluer le score du modèle sur le jeu de test
- Récupérer les différentes métriques du modèle pour pouvoir le comparer aux autres

### 2.2.1: Jeu sans transformation

In [14]:
listeModeles.append('regression_lineaire')

Pour optimiser la méthode de régression linéaire, il faut sélectionner les features qui sont corrélées avec la target mais ne sont pas corrélées entre elles. Pour cela, on peut utiliser un algorithme de sélection des x meilleures features, où x est un hyperparamètre que l'on peut trouver par une recherche sur grille.

In [15]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
r2 = []
nb_features = range(1, X_train_std.shape[1])

for i in nb_features:
    f_selector = SelectKBest(score_func=f_regression, k=i)
    f_selector.fit(X_train_std, y_train)
    X_train_fs = f_selector.transform(X_train_std)
    lm = LinearRegression()
    r2.append(cross_val_score(lm, X_train_fs, y_train).mean())

r2_max = max(r2)
k = r2.index(r2_max)+1
print("Le meilleur coefficient r2 = " + str(r2_max) + " est obtenu en sélectionnant les " + str(k) + " meilleures features")

Le meilleur coefficient r2 = 0.8720961045980872 est obtenu en sélectionnant les 2 meilleures features


In [16]:
#Cross-validation du modèle (avec les paramètres optimaux)
lm = LinearRegression()
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_train_std, y_train)
X_train_fs = f_selector.transform(X_train_std)
print(cross_val_score(lm, X_train_fs, y_train,  cv=5))

[0.89531487 0.86659288 0.90425655 0.85607833 0.83823789]


In [17]:
#Entrainement du modèle sur le training_set
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_train_std, y_train)
X_train_fs = f_selector.transform(X_train_std)
X_test_fs = f_selector.transform(X_test_std)

start_time = timeit.default_timer()
lm = LinearRegression().fit(X_train_fs, y_train)
y_predict = lm.predict(X_test_fs)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lm.score(X_train_fs, y_train)))
listeR2_train.append(lm.score(X_train_fs, y_train))

r2_score train: 0.9057


In [18]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 61955901846097.7812
r2_score: 0.7640
Temps de calcul: 0.0013


In [19]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

On note de l'instabilité lors de la validation croisée et un soucis de sur-apprentissage sur le jeu d'entraînement.

### 2.2.2: Jeu avec transformation

In [20]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
r2 = []
nb_features = range(1, X_log_train_std.shape[1])

for i in nb_features:
    f_selector = SelectKBest(score_func=f_regression, k=i)
    f_selector.fit(X_log_train_std, y_log_train)
    X_log_train_fs = f_selector.transform(X_log_train_std)
    lm = LinearRegression()
    r2.append(cross_val_score(lm, X_log_train_fs, y_log_train).mean())

r2_max = max(r2)
k = r2.index(r2_max)+1
print("Le meilleur coefficient r2 = " + str(r2_max) + " est obtenu en sélectionnant les " + str(k) + " meilleures features")

Le meilleur coefficient r2 = 0.6247041542249571 est obtenu en sélectionnant les 17 meilleures features


In [21]:
#Cross-validation du modèle (avec les paramètres optimaux)
lm = LinearRegression()
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_log_train_std, y_log_train)
X_log_train_fs = f_selector.transform(X_log_train_std)
print(cross_val_score(lm, X_log_train_fs, y_log_train,  cv=5))

[0.55766597 0.60548773 0.62653678 0.71570915 0.61812113]


In [22]:
#Entrainement du modèle sur le training_set
f_selector = SelectKBest(score_func=f_regression, k=k)
f_selector.fit(X_log_train_std, y_log_train)
X_log_train_fs = f_selector.transform(X_log_train_std)
X_log_test_fs = f_selector.transform(X_log_test_std)

start_time = timeit.default_timer()
lm = LinearRegression().fit(X_log_train_fs, y_log_train)
y_log_predict = lm.predict(X_log_test_fs)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lm.score(X_log_train_fs, y_log_train)))
listeR2_log_train.append(lm.score(X_log_train_fs, y_log_train))

r2_score train: 0.6273


In [23]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.5494
r2_score: 0.7066
Temps de calcul: 0.0083


In [24]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Les transformations logarithmiques n'améliorent pas le score de prédiciton.

## <a name="C5">2.3: Régression Ridge</a>
Pour éviter l'éventuel sur-appretissage du modèle de régression linéaire, on va utiliser une technique de régularisation afin de pouvoir contrôler en plus de l'erreur du modèle, sa complexité.
### 2.3.1: Jeu sans transformation

In [25]:
listeModeles.append("regression_ridge")

In [26]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_train_std, y_train)
alpha = reg.alpha_

In [27]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmr = Ridge(alpha)
print(cross_val_score(lmr, X_train_std, y_train,  cv=5))

[0.88381533 0.85169902 0.87339425 0.8368557  0.81610432]


In [28]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmr = Ridge(alpha).fit(X_train_std, y_train)
y_predict = lmr.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmr.score(X_train_std, y_train)))
listeR2_train.append(lmr.score(X_train_std, y_train))

r2_score train: 0.9244


In [29]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 60302742798950.3359
r2_score: 0.7703
Temps de calcul: 0.0358


In [30]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Bon score mais avec du sur-apprentissage.

### 2.3.2: Jeu avec transformations

In [31]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_log_train_std, y_log_train)
alpha = reg.alpha_

In [32]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmr = Ridge(alpha)
print(cross_val_score(lmr, X_log_train_std, y_log_train,  cv=5))

[0.59486996 0.6162871  0.56954364 0.73823023 0.62494982]


In [33]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmr = Ridge(alpha).fit(X_log_train_std, y_log_train)
y_log_predict = lmr.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmr.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lmr.score(X_log_train_std, y_log_train))

r2_score train: 0.6943


In [34]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.6618
r2_score: 0.6466
Temps de calcul: 0.0386


In [35]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Plus stable mais score moins bon.

## <a name="C6">2.4: Régression Lasso</a>
Pour tenter d'améliorer la capacité de généralisation de la régression ridge on peut ajouter en plus une réduction dimensionnelle, notamment pour supprimer les éventuelles variables corrélées.
### 2.4.1: Jeu sans transformation:

In [36]:
listeModeles.append('regression_lasso')

In [37]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = LassoCV(cv=5, random_state=0).fit(X_train_std, y_train)
alpha = reg.alpha_

In [38]:
#Cross-validation du modèle (avec les paramètres optimaux)
lml = Lasso(alpha)
print(cross_val_score(lml, X_train_std, y_train,  cv=5))

[0.89219645 0.86847906 0.89611824 0.85481584 0.81485454]


In [39]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lml = Lasso(alpha).fit(X_train_std, y_train)
y_predict = lml.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lml.score(X_train_std, y_train)))
listeR2_train.append(lml.score(X_train_std, y_train))

r2_score train: 0.9205


In [40]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 59703455711659.5156
r2_score: 0.7726
Temps de calcul: 0.0688


In [41]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Encore un soucis de sur-apprentissage.

### 2.4.2: Jeu avec transformations

In [42]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = LassoCV(cv=5, random_state=0).fit(X_log_train_std, y_log_train)
alpha = reg.alpha_

In [43]:
#Cross-validation du modèle (avec les paramètres optimaux)
lml = Lasso(alpha)
print(cross_val_score(lml, X_log_train_std, y_log_train,  cv=5))

[0.5851821  0.62840481 0.58832619 0.7376808  0.62799986]


In [44]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lml = Lasso(alpha).fit(X_log_train_std, y_log_train)
y_log_predict = lml.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lml.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lml.score(X_log_train_std, y_log_train))

r2_score train: 0.6817


In [45]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.6271
r2_score: 0.6651
Temps de calcul: 0.0926


In [46]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

## <a name="C7">2.5: Régression ElasticNet</a>
La régression ridge permet d'éviter le sur-appretissage avec une solution unique lorsque la régression lasso permet d'avoir un modèle plus parcimonieux mais instable. Il exite un modèle de prédiction linéaire intermédiaire qui combine les deux approches: la régression ElasticNet.
### 2.5.1: Jeu sans transformation

In [47]:
listeModeles.append('regression_elasticNet')

In [48]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .9], cv=5).fit(X_train_std, y_train)
alpha = reg.alpha_
l1_ratio = reg.l1_ratio_

In [49]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
print(cross_val_score(lml, X_train_std, y_train,  cv=5))

C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.0899696486248616e+16, tolerance: 167198021309764.5
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.070738451199455e+16, tolerance: 168761484656366.56
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.20022078673733e+16, tolerance: 169336538221690.6
  model = cd_fast.enet_coordinate_descent(
C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conv

[0.8832066  0.84956832 0.87178261 0.83621119 0.81782764]


C:\Users\vbour\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4401473099411936e+16, tolerance: 64319035816794.4
  model = cd_fast.enet_coordinate_descent(


In [50]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio).fit(X_train_std, y_train)
y_predict = lmE.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmE.score(X_train_std, y_train)))
listeR2_train.append(lmE.score(X_train_std, y_train))

r2_score train: 0.0024


In [51]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 261999458004006.6562
r2_score: 0.0021
Temps de calcul: 0.0396


In [52]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Instabilité propre à la régression lasso.

### 2.5.2: Jeu avec transformations

In [53]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
reg = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .9], cv=5).fit(X_log_train_std, y_log_train)
alpha = reg.alpha_
l1_ratio = reg.l1_ratio_

In [54]:
#Cross-validation du modèle (avec les paramètres optimaux)
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
print(cross_val_score(lmE, X_log_train_std, y_log_train,  cv=5))

[0.58400565 0.63050978 0.58951621 0.73805939 0.62623544]


In [55]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
lmE = ElasticNet(alpha=alpha, l1_ratio=l1_ratio).fit(X_log_train_std, y_log_train)
y_log_predict = lmE.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(lmE.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(lmE.score(X_log_train_std, y_log_train))

r2_score train: 0.6813


In [56]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.6251
r2_score: 0.6662
Temps de calcul: 0.1122


In [57]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

# <a name="C8">Partie 3: Méthodes ensemblistes</a>
Le principe des méthodes ensemblistes est de combiner un certain nombre de modèles avec des performances faibles pour obtenir un modèle prédictif plus efficace en combinant les résultats via la moyenne par exemple pour les problèmes de régression.
## <a name="C9">3.1: Bagging</a>
Le bagging fonctionne en 3 étapes:
- On génère N nouveaux jeux de données en tirant aléatoirement (avec remise) chaque observation dans le jeu de données initial
- On entraîne un apprenant faible sur chaque jeu, ici un arbre de décision
- On effectue une prédiction en faisant la moyenne des prédictions des apprenants faibles
### 3.1.1: Jeu sans transformation

In [58]:
listeModeles.append('Bagging')

In [59]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100]}
reg = GridSearchCV (estimator=BaggingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
n_estimators = reg.best_params_.get('n_estimators')

In [60]:
#Cross-validation du modèle (avec les paramètres optimaux)
bagging = BaggingRegressor(n_estimators=n_estimators)
print(cross_val_score(bagging, X_train_std, y_train,  cv=5))

[0.84804425 0.84778657 0.92099902 0.9247515  0.57114665]


In [61]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
bagging = BaggingRegressor(n_estimators=n_estimators).fit(X_train_std, y_train)
y_predict = bagging.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(bagging.score(X_train_std, y_train)))
listeR2_train.append(bagging.score(X_train_std, y_train))

r2_score train: 0.9826


In [62]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 34570920842303.6680
r2_score: 0.8683
Temps de calcul: 1.1578


In [63]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Instabilité et sur-apprentissage.

### 3.1.2: Jeu avec transformations

In [64]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100]}
reg = GridSearchCV (estimator=BaggingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
n_estimators = reg.best_params_.get('n_estimators')

In [65]:
#Cross-validation du modèle (avec les paramètres optimaux)
bagging = BaggingRegressor(n_estimators=n_estimators)
print(cross_val_score(bagging, X_log_train_std, y_log_train,  cv=5))

[0.76510641 0.61400888 0.80466126 0.72960754 0.8084001 ]


In [66]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
bagging = BaggingRegressor(n_estimators=n_estimators).fit(X_log_train_std, y_log_train)
y_log_predict = bagging.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(bagging.score(X_log_train_std, y_log_train)))
listeR2_log_train.append(bagging.score(X_log_train_std, y_log_train))

r2_score train: 0.9669


In [67]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.3174
r2_score: 0.8305
Temps de calcul: 11.0795


In [68]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Résultats similaires avec ou sans la variable `ENERGYSTARScore`.

## <a name="C10">3.2: Random Forest</a>
Les forêts aléatoires reposent sur le même principe que le bagging mais en sélectionnant un nombre plus restreint de features afin de créer des arbres les plus différents possible les uns des autres et limiter ainsi le sur-apprentissage.
### 3.2.1: Jeu sans transformation

In [69]:
listeModeles.append('RandomForest')

In [70]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'max_depth': [10, 25, 50, 100]}
reg = GridSearchCV (estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
n_estimators = reg.best_params_.get('n_estimators')
max_depth = reg.best_params_.get('max_depth')

In [71]:
#Cross-validation du modèle (avec les paramètres optimaux)
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
print(cross_val_score(forest, X_train_std, y_train,  cv=5))

[0.93489615 0.89320569 0.92494715 0.93330945 0.57184434]


In [72]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_train_std, y_train)
y_predict = forest.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(forest.score(X_train_std, y_train))) 
listeR2_train.append(forest.score(X_train_std, y_train))

r2_score train: 0.9265


In [73]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 34463818195127.7109
r2_score: 0.8687
Temps de calcul: 6.0335


In [74]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Un petit peu d'instabilité et de sur-apprentissage.

### 3.2.2: Jeu avec transformations

In [75]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'max_depth': [10, 25, 50, 100]}
reg = GridSearchCV (estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
n_estimators = reg.best_params_.get('n_estimators')
max_depth = reg.best_params_.get('max_depth')

In [76]:
#Cross-validation du modèle (avec les paramètres optimaux)
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
print(cross_val_score(forest, X_log_train_std, y_log_train,  cv=5))

[0.74398959 0.6257683  0.8141205  0.71765055 0.80505023]


In [77]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_log_train_std, y_log_train)
y_log_predict = forest.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(forest.score(X_log_train_std, y_log_train))) 
listeR2_log_train.append(forest.score(X_log_train_std, y_log_train))

r2_score train: 0.9681


In [78]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.2874
r2_score: 0.8465
Temps de calcul: 8.2716


In [79]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Instabilité et sur-apprentissage.

## <a name="C11">3.3: Gradient Boosting</a>
Les méthodes ensemblistes utilisées plus haut sont dites parallèles, c'est-à-dire que les apprenants faibles sont entraînés indépendamment les uns des autres. Le Gradient Boosting au contraire est une méthode ensembliste dite séquentielle. Les apprenants faibles sont entraînés les uns à la suite des autres, notamment en mettant un peu plus l'accent sur les observations qui ont provoqué des erreurs sur l'apprenant précédent.
### 3.3.1: Jeu sans transformation

In [80]:
listeModeles.append('GradientBoosting')

In [81]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'learning_rate': [0.01, 0.1, 0.2, 0.3]}
reg = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
n_estimators = reg.best_params_.get('n_estimators')
learning_rate = reg.best_params_.get('learning_rate')

In [82]:
#Cross-validation du modèle (avec les paramètres optimaux)
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
print(cross_val_score(boosting, X_train_std, y_train,  cv=5))

[0.93312085 0.90389181 0.93941667 0.9538519  0.60022129]


In [83]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0).fit(X_train_std, 
                                                                                                               y_train)
y_predict = boosting.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(boosting.score(X_train_std, y_train))) 
listeR2_train.append(boosting.score(X_train_std, y_train))

r2_score train: 0.9931


In [84]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 32826867892228.5625
r2_score: 0.8750
Temps de calcul: 2.9462


In [85]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Un petit peu d'instabilité et du sur-apprentissage.

### 3.3.2: Jeu avec transformations

In [86]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'n_estimators': [10, 50, 100],
         'learning_rate': [0.01, 0.1, 0.2, 0.3]}
reg = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
n_estimators = reg.best_params_.get('n_estimators')
learning_rate = reg.best_params_.get('learning_rate')

In [87]:
#Cross-validation du modèle (avec les paramètres optimaux)
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
print(cross_val_score(boosting, X_log_train_std, y_log_train,  cv=5))

[0.79654621 0.64988404 0.80893843 0.74488517 0.88002611]


In [88]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
boosting = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0).fit(X_log_train_std, 
                                                                                                               y_log_train)
y_log_predict = boosting.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(boosting.score(X_log_train_std, y_log_train))) 
listeR2_log_train.append(boosting.score(X_log_train_std, y_log_train))

r2_score train: 0.8962


In [89]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.3051
r2_score: 0.8371
Temps de calcul: 2.0297


In [90]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

Instabilité.

# <a name="C12">Partie 4: Méthodes non linéaires</a>
Beaucoup de phénomènes n'étant pas linéaires, on peut se retrouver dans une impasse si l'on cherche à effectuer des prédictions à l'aide de méthodes linéaires comme celles utilisées jusqu'à maintenant. On va ainsi tester des méthodes non linéaires pour voir si l'on peut obtenir de meilleures performances de généralisation.
## <a name="C13">4.1: SVM à noyau</a>
### 4.1.1: Jeu sans transformation

In [91]:
listeModeles.append('SVM_noyau')

In [92]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'C': np.logspace(-2, 3, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=SVR(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
C = reg.best_params_.get('C')
gamma = reg.best_params_.get('gamma')

In [93]:
#Cross-validation du modèle (avec les paramètres optimaux)
svr = SVR(kernel='rbf', C=C, gamma=gamma)
print(cross_val_score(svr, X_train_std, y_train,  cv=5))

[-0.04384446 -0.04710484 -0.04637958 -0.0533015  -0.020493  ]


In [94]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
svr = SVR(kernel='rbf', C=C, gamma=gamma).fit(X_train_std, y_train)
y_predict = svr.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(svr.score(X_train_std, y_train))) 
listeR2_train.append(svr.score(X_train_std, y_train))

r2_score train: -0.0274


In [95]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 273902099379896.0000
r2_score: -0.0433
Temps de calcul: 11.9544


In [96]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

### 4.1.2: Jeu avec Transformations

In [97]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'C': np.logspace(-2, 3, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=SVR(), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
C = reg.best_params_.get('C')
gamma = reg.best_params_.get('gamma')

In [98]:
#Cross-validation du modèle (avec les paramètres optimaux)
svr = SVR(kernel='rbf', C=C, gamma=gamma)
print(cross_val_score(svr, X_log_train_std, y_log_train,  cv=5))

[0.50002277 0.5878945  0.55374873 0.65748647 0.55689429]


In [99]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
svr = SVR(kernel='rbf', C=C, gamma=gamma).fit(X_log_train_std, y_log_train)
y_log_predict = svr.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(svr.score(X_log_train_std, y_log_train))) 
listeR2_log_train.append(svr.score(X_log_train_std, y_log_train))

r2_score train: 0.6686


In [100]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 0.6185
r2_score: 0.6697
Temps de calcul: 7.2911


In [101]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

On obtient de meilleurs résultats avec les transformations logarithmiques même si le modèle reste instable.

## <a name="C14">4.2: Régression ridge à noyau</a>
Le principe de fonctionnement est le même que pour la SVM à noyau, seule la fonction de perte utilisée diffère.
### 4.2.1: Jeu sans transformation

In [102]:
listeModeles.append('Ridge_noyau')

In [103]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'alpha': np.logspace(-2, 2, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=KernelRidge(kernel='rbf'), param_grid=params, cv=5, scoring='r2')
reg.fit(X_train_std, y_train)
alpha = reg.best_params_.get('alpha')
gamma = reg.best_params_.get('gamma')

In [104]:
#Cross-validation du modèle (avec les paramètres optimaux)
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma)
print(cross_val_score(krr, X_train_std, y_train,  cv=5))

[0.91661739 0.74557024 0.80588276 0.79190096 0.11636417]


In [105]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma).fit(X_train_std, y_train)
y_predict = krr.predict(X_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(krr.score(X_train_std, y_train))) 
listeR2_train.append(krr.score(X_train_std, y_train))

r2_score train: 0.9981


In [106]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score: {:.4f}".format(r2_score(y_test, y_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 59467639647582.5938
r2_score: 0.7735
Temps de calcul: 5.5905


In [107]:
#Enregistrement des métriques
listeMSE.append(round(mean_squared_error(y_test, y_predict),4))
listeR2.append(round(r2_score(y_test, y_predict),4))
listeTemps.append(round(time,4))

Modèle instable et sur-apprentissage.

### 4.2.2: Jeu avec transformations

In [108]:
#GridSearch pour trouver le(s) paramètre(s) qui renvoie(nt) le meilleur score R²
params = {'alpha': np.logspace(-2, 2, 3),
         'gamma': np.logspace(-2, 1, 3)
}
reg = GridSearchCV (estimator=KernelRidge(kernel='rbf'), param_grid=params, cv=5, scoring='r2')
reg.fit(X_log_train_std, y_log_train)
alpha = reg.best_params_.get('alpha')
gamma = reg.best_params_.get('gamma')

In [109]:
#Cross-validation du modèle (avec les paramètres optimaux)
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma)
print(cross_val_score(krr, X_log_train_std, y_log_train,  cv=5))

[-2.39195938 -2.54296061 -2.38484026 -2.062677   -1.56570367]


In [110]:
#Entrainement du modèle sur le training_set
start_time = timeit.default_timer()
krr = KernelRidge(kernel='rbf', alpha=alpha, gamma=gamma).fit(X_log_train_std, y_log_train)
y_log_predict = krr.predict(X_log_test_std)
time = timeit.default_timer() - start_time

#Score sur le jeu d'entraînement
print("r2_score train: {:.4f}".format(krr.score(X_log_train_std, y_log_train))) 
listeR2_log_train.append(krr.score(X_log_train_std, y_log_train))

r2_score train: 0.8828


In [111]:
#Score du modèle sur le test_set
print("Mean squared error: {:.4f}".format(mean_squared_error(y_log_test, y_log_predict)))
print("r2_score: {:.4f}".format(r2_score(y_log_test, y_log_predict)))
print("Temps de calcul: {:.4f}".format(time))

Mean squared error: 4.7763
r2_score: -1.5507
Temps de calcul: 6.1542


In [112]:
#Enregistrement des métriques
listeMSE_log.append(round(mean_squared_error(y_log_test, y_log_predict),4))
listeR2_log.append(round(r2_score(y_log_test, y_log_predict),4))
listeTemps_log.append(round(time,4))

résultats médiocres.

# <a name="C15">Partie 5: Sélection du modèle de prédiction</a>
Il est maintenant temps de choisir parmis les modèles testés, lequel semble le plus pertinent et adapté pour prédire les émissions de CO2 des bâtiments de la ville de Seattle. Notre choix s'appuiera sur la capacité de généralisation du modèle, son temps de calcul et les contraintes liées à la sélection des hyperparamètres.

In [113]:
#Jeu non transformé
tab = np.array([listeMSE, listeR2_train, listeR2, listeTemps])
df = pd.DataFrame(tab, index=['MSE', 'r2 train', 'r2 test', 'Temps_Calcul'], columns=listeModeles)
df

,Dummy_mean,Dummy_median,regression_lineaire,regression_ridge,regression_lasso,regression_elasticNet,Bagging,RandomForest,GradientBoosting,SVM_noyau,Ridge_noyau
MSE,2.626130e+14,2.752834e+14,6.195590e+13,6.030274e+13,5.970346e+13,2.619995e+14,3.457092e+13,3.446382e+13,3.282687e+13,2.739021e+14,5.946764e+13
r2 train,0.000000e+00,-3.112603e-02,9.057054e-01,9.244434e-01,9.204933e-01,2.400787e-03,9.825816e-01,9.265382e-01,9.930764e-01,-2.736532e-02,9.980674e-01
r2 test,-3.000000e-04,-4.850000e-02,7.640000e-01,7.703000e-01,7.726000e-01,2.100000e-03,8.683000e-01,8.687000e-01,8.750000e-01,-4.330000e-02,7.735000e-01
Temps_Calcul,2.200000e-03,9.000000e-04,1.300000e-03,3.580000e-02,6.880000e-02,3.960000e-02,1.157800e+00,6.033500e+00,2.946200e+00,1.195440e+01,5.590500e+00


In [114]:
#Jeu transformé
tab = np.array([listeMSE_log, listeR2_log_train, listeR2_log, listeTemps_log])
df_log = pd.DataFrame(tab, index=['MSE', 'r2 train', 'r2 test', 'Temps_Calcul'], columns=listeModeles)
df_log

,Dummy_mean,Dummy_median,regression_lineaire,regression_ridge,regression_lasso,regression_elasticNet,Bagging,RandomForest,GradientBoosting,SVM_noyau,Ridge_noyau
MSE,1.8750,1.901100,0.549400,0.661800,0.627100,0.625100,0.317400,0.287400,0.305100,0.618500,4.776300
r2 train,0.0000,-0.007161,0.627319,0.694333,0.681658,0.681273,0.966899,0.968068,0.896245,0.668586,0.882772
r2 test,-0.0013,-0.015300,0.706600,0.646600,0.665100,0.666200,0.830500,0.846500,0.837100,0.669700,-1.550700
Temps_Calcul,0.0010,0.001300,0.008300,0.038600,0.092600,0.112200,11.079500,8.271600,2.029700,7.291100,6.154200


Les transformations logarithmiques n'apportent pas de gain sur les scores de prédiction et n'améliorent pas la robustesse des modèles comme ce fut le cas lors de la prédiciton des émissions de CO2. On préfèrerra donc conserver le jeu sans transformations. 

Les modèles les plus intéressants sont les méthodes ensemblistes.</br>
Le bagging avait tendance à être plus instable que les deux autres. On va donc l'écarter.</br>
Les fôrets aléatoire souffrent moins de sur-apprentissage mais ont un temps d'exécution assez long sans pour autant gagner en précision. On écarte également ce modèle.

Finalement, on fait le choix du **Gradient Boosting** sur le jeu de données non transformé qui donne de bons résultats avec un temps d'exécution assez court. Attention toutefois à un petit peu d'instabilité. </br>
Pour tenter d'améliorer encore plus ce modèle, on peut essayer d'optimiser les deux hyperparamètres **n_estimators** et **learning_rate** sur des plages de valeurs plus larges, notamment pour le nombre d'apprenants faibles **n_estimators**.